# LlamaParse Agent document parser example
OpenAI Agent, Document parsing using LlamaParse.

Please set your OpenAI API key and AgentOps key in a local .env file for load_dotenv():
```
OPENAI_API_KEY=sk...
AGENTOPS_API_KEY=
LLAMA_CLOUD_API_KEY=
```
or set it in the environment explicity using 
```
import os
os.environ["OPENAI_API_KEY"] = "sk..."
os.environ["AGENTOPS_API_KEY"] = ".."
os.environ["LLAMA_CLOUD_API_KEY"] = ".."
```

In [1]:
!pip install llama-parse llama-index llama-index-postprocessor-sbert-rerank



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.2)

In [7]:
# swe agents paper: https://arxiv.org/abs/2405.15793
!wget https://arxiv.org/pdf/2405.15793.pdf -O paper_1.pdf

--2024-11-17 16:44:05--  https://arxiv.org/pdf/2405.15793.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.3.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2405.15793 [following]
--2024-11-17 16:44:05--  http://arxiv.org/pdf/2405.15793
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4963538 (4.7M) [application/pdf]
Saving to: ‘paper_1.pdf’

paper_1.pdf         100%[===================>]   4.73M  17.1MB/s    in 0.3s    

2024-11-17 16:44:05 (17.1 MB/s) - ‘paper_1.pdf’ saved [4963538/4963538]



In [8]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",
)

In [9]:
documents = await parser.aload_data("paper_1.pdf")

Started parsing the file under job_id b16749c7-1853-494f-b212-70c939c71244
........................

In [21]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.node_parser import (
    MarkdownElementNodeParser,
    SentenceSplitter,
)

# explicitly extract tables with the MarkdownElementNodeParser
node_parser = MarkdownElementNodeParser(num_workers=8)
nodes = node_parser.get_nodes_from_documents(documents)
nodes, objects = node_parser.get_nodes_and_objects(nodes)

# Chain splitters to ensure chunk size requirements are met
nodes = SentenceSplitter(chunk_size=512, chunk_overlap=20).get_nodes_from_documents(
    nodes
)

1it [00:00, 19508.39it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
3it [00:00, 44620.26it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 38836.15it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 25575.02it/s]
2it [00:00, 29433.71it/s]
2it [00:00, 31184.42it/s]
1it [00:00, 13189.64it/s]
2it [00:00, 28435.96it/s]
1it [00:00, 16578.28it/s]
1it [00:00, 13530.01it/s]
0it [00:00, ?it/s]
2it [00:00, 27324.46it/s]
2it [00:00, 57065.36it/s]
3it [00:00, 30247.38it/s]
1it [00:00, 14716.86it/s]
1it [00:00, 13357.66it/s]
1it [00:00, 14315.03it/s]
1it [00:00, 16710.37it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00

In [22]:
from llama_index.core import VectorStoreIndex, SummaryIndex

vector_index = VectorStoreIndex(nodes=nodes)
summary_index = SummaryIndex(nodes=nodes)

In [13]:
!pip install llama-index-postprocessor-colbert-rerank

  Using cached llama_index_core-0.12.0-py3-none-any.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 26.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 31.4 MB/s eta 0:00:0000:010:01
  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
  Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 23.6 MB/s eta 0:00:00
  Using cached safetensors-0.4.5-cp311-cp311-macosx_11_0_arm64.whl (381 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.23
    Uninstalling llama-index-core-0.11.23:
      Successfully uninstalled llama-index-core-0.11.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the foll

In [24]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.postprocessor.colbert_rerank import ColbertRerank

tools = [
    QueryEngineTool(
        vector_index.as_query_engine(
            similarity_top_k=8, node_postprocessors=[ColbertRerank(top_n=3)]
        ),
        metadata=ToolMetadata(
            name="search",
            description="Search the document, pass the entire user message in the query",
        ),
    ),
    QueryEngineTool(
        summary_index.as_query_engine(),
        metadata=ToolMetadata(
            name="summarize",
            description="Summarize the document using the user message",
        ),
    ),
]

agent = OpenAIAgent.from_tools(tools=tools, verbose=True)

In [25]:
# Takes long time.
resp = agent.chat("What is the summary of the paper?")

Added user message to memory: What is the summary of the paper?


In [26]:
print(str(resp))

Sure, please provide the text of the paper so I can generate a summary for you.


In [ ]:
resp = agent.chat("How do the authors evaluate their work?")

In [ ]:
print(str(resp))